In [35]:
from alpaca_functions import *
import alpaca_functions
import stonk_pull 
import signals
import datetime
from binance2 import *
import binance2
import json
from datetime import date
from time import mktime
import time
import urllib3
import urllib.request
import math
from dotenv import load_dotenv


In [2]:
# This simulates holding a current bank account that can send and receive
# money via ACH transfers to and from Alpaca
# We are assuming we have 100,000 in the bank and we are sending 100,000 to our alpaca algorithm

#We will be using STRIPE as our shared bank account, see stripe.py 

beginning_deposit = 100000

bank_balance = 100000

send_to_alpaca_amount = 100000

withdraw_from_alpaca_amount = 0

binance_balance = 0

profits = 0

In [3]:
#load alpaca api keys
api = stonk_pull.load()


Python-dotenv could not parse statement starting at line 11
Python-dotenv could not parse statement starting at line 15
C:\Users\costa\desktop\byte\rice\ru-hou-fin-pt-03-2020-u-c\Project3_DoH\Final\stonk_pull.py:16: UserWarning: Make sure your api keys are renamed to fit this function or alter the function to fit your names
  warnings.warn('Make sure your api keys are renamed to fit this function or alter the function to fit your names')


In [4]:
# Send money to alpaca. This function calculates a new bank balance after we send money to alpaca
# In reality this would be an ACH transfer from our bank to alpaca.
bank_balance = send_to_alpaca(send_to_alpaca_amount, bank_balance)
bank_balance


Transfer to alpaca successful. Your bank balance is now 0 dollars



0

In [5]:
#Pull the data from alphavantage
data = stonk_pull.data_pull(api)
data.head()

,SPY Open,SPY High,SPY Low,SPY Close,SPY Adjusted Close,SPY Volume,SPY Dividend Amount,SPY Split Coefficient
date,,,,,,,,
2020-08-24,342.12,343.00,339.4504,342.92,342.92,48588662.0,0.0,1.0
2020-08-21,337.92,339.72,337.5500,339.48,339.48,55106628.0,0.0,1.0
2020-08-20,335.36,338.80,335.2200,338.28,338.28,42207826.0,0.0,1.0
2020-08-19,339.05,339.61,336.6200,337.23,337.23,68054244.0,0.0,1.0
2020-08-18,338.34,339.10,336.6100,338.64,338.64,38733908.0,0.0,1.0


In [6]:
#Clean the data
stock_data_df = stonk_pull.data_clean(data)
stock_data_df.head()


,SPY Open,SPY Close,SPY Volume
date,,,
2020-08-24,342.12,342.92,48588662.0
2020-08-21,337.92,339.48,55106628.0
2020-08-20,335.36,338.28,42207826.0
2020-08-19,339.05,337.23,68054244.0
2020-08-18,338.34,338.64,38733908.0


In [7]:
#Sort earliest to latest.
stock_data_df.sort_index(inplace=True, ascending=True)
stock_data_df.head()


,SPY Open,SPY Close,SPY Volume
date,,,
1999-11-01,136.5000,135.5625,4006500.0
1999-11-02,135.9687,134.5937,6516900.0
1999-11-03,136.0000,135.5000,7222300.0
1999-11-04,136.7500,136.5312,7907500.0
1999-11-05,138.6250,137.8750,7431500.0


In [8]:
#To set up the crossover strategy select the one column we need, "Close", and set to dataframe
signals_df = stock_data_df['SPY Close'].to_frame()
signals_df.head()

,SPY Close
date,
1999-11-01,135.5625
1999-11-02,134.5937
1999-11-03,135.5000
1999-11-04,136.5312
1999-11-05,137.8750


In [9]:
#This function generates the signals dataframe
signals.generate_sma2_sma13_signals(signals_df)

,SPY Close,SMA2,SMA13,Signal,Buy/Sell
date,,,,,
1999-11-01,135.5625,NaN,NaN,0.0,NaN
1999-11-02,134.5937,135.07810,NaN,0.0,0.0
1999-11-03,135.5000,135.04685,NaN,0.0,0.0
1999-11-04,136.5312,136.01560,NaN,0.0,0.0
1999-11-05,137.8750,137.20310,NaN,0.0,0.0
...,...,...,...,...,...
2020-08-18,338.6400,338.27500,334.031538,1.0,0.0
2020-08-19,337.2300,337.93500,334.855385,1.0,0.0
2020-08-20,338.2800,337.75500,335.585385,1.0,0.0


In [10]:
#Send Buy and Sell orders to alpaca using the signal, 1 = buy, -1 = sell, 0 = hold

if signals_df['Buy/Sell'].iloc[-1] == 1.0:
    symbol = "SPY"
    alpaca_functions.buy(symbol)
    
elif signals_df['Buy/Sell'].iloc[-1] == -1.0:
    symbol = "SPY"
    alpaca_functions.sell_all(symbol)
    
elif signals_df['Buy/Sell'].iloc[-1] == 0.0:
    print("The signal is hold. No action taken")

print(f"Current positions are: {api.list_positions()}")

The signal is hold. No action taken
Current positions are: [Position({   'asset_class': 'us_equity',
    'asset_id': 'b28f4066-5c6d-479b-a2af-85dc1a8f16fb',
    'avg_entry_price': '343.2',
    'change_today': '0.0022162603522688',
    'cost_basis': '99871.2',
    'current_price': '343.68',
    'exchange': 'ARCA',
    'lastday_price': '342.92',
    'market_value': '100010.88',
    'qty': '291',
    'side': 'long',
    'symbol': 'SPY',
    'unrealized_intraday_pl': '139.68',
    'unrealized_intraday_plpc': '0.0013986013986014',
    'unrealized_pl': '139.68',
    'unrealized_plpc': '0.0013986013986014'})]


In [11]:
#On the first of the month we will send our profits from alpaca to the bank

day_of_month = (date.today()).strftime("%d")

if day_of_month == '1':
    bank_balance = transfer_profits_from_alpaca(bank_balance, beginning_deposit)
else:
    print("No profits were transferred to the bank from alpaca today.")  
    



Waiting...
Your transfer of profits from alpaca was successful. Your new bank balance is $891.9499999999971


In [12]:
bank_balance

891.9499999999971

In [13]:
#On the second day of the month we will send our profits from our bank to the binance cryptoexchange

day_of_month = (date.today()).strftime("%d")

#If the the day of the month is 2, we transfer profits to binance and recalculate the bank balance and binance balance

if day_of_month == '2':
    #transfer function returns a list with the new bank balance and the new binance balance
    balances = transfer_profits_to_exchange(bank_balance, binance_balance)
    
    #set the new bank and the new binance balances
    bank_balance = balances[0]
    binance_balance = balances[1]
    
else:
    print("No profits were transferred from the bank to binance today.")  

print("bank balance:", bank_balance)
print("binance balance:", binance_balance)



Transfer to Binance successful. Your binance_balance is now 891.9499999999971.
Your bank balance is now 0.
bank balance: 0
binance balance: 891.9499999999971


In [14]:
# Get price for ETH through binance api
eth_price = get_eth_price()
eth_price = float(eth_price)
eth_price

407.4

In [15]:


#Calculate number of ether to buy, round down to nearest integer because binance does not support fractional buys
quantity = math.floor(binance_balance/eth_price)
quantity


2

In [45]:
#This buys shares of ETH 

buy_ETH(quantity)

b'{"symbol":"ETHBUSD","orderId":46,"orderListId":-1,"clientOrderId":"my_order_id_1","transactTime":1598383158268}'


b'{"symbol":"ETHBUSD","orderId":46,"orderListId":-1,"clientOrderId":"my_order_id_1","transactTime":1598383158268}'

In [46]:
#check account holdings
holdings = get_account_holdings()

#convert bytes object to string
holdings = holdings.decode("utf-8") 
    
#convert str to json
print(f"This is your account information and current holdings")
holdings = json.loads(holdings)
holdings

This is your account information and current holdings


{'makerCommission': 0,
 'takerCommission': 0,
 'buyerCommission': 0,
 'sellerCommission': 0,
 'canTrade': True,
 'canWithdraw': False,
 'canDeposit': False,
 'updateTime': 1598383158268,
 'accountType': 'SPOT',
 'balances': [{'asset': 'BNB',
   'free': '1000.00000000',
   'locked': '0.00000000'},
  {'asset': 'BTC', 'free': '0.99000000', 'locked': '0.00000000'},
  {'asset': 'BUSD', 'free': '5558.07298600', 'locked': '0.00000000'},
  {'asset': 'ETH', 'free': '110.90311000', 'locked': '0.00000000'},
  {'asset': 'LTC', 'free': '500.00000000', 'locked': '0.00000000'},
  {'asset': 'TRX', 'free': '500000.00000000', 'locked': '0.00000000'},
  {'asset': 'USDT', 'free': '10400.00000000', 'locked': '0.00000000'},
  {'asset': 'XRP', 'free': '50000.00000000', 'locked': '0.00000000'}],
 'permissions': ['SPOT']}

In [47]:
#check recent trades to see if trade went through
recent_trades = json.loads((check_recent_trades()).decode('utf-8'))
recent_trades

b'[{"id":0,"price":"386.26000000","qty":"1.00000000","quoteQty":"386.26000000","time":1596440526404,"isBuyerMaker":true,"isBestMatch":true},{"id":1,"price":"414.15000000","qty":"0.02714000","quoteQty":"11.24003100","time":1597851105665,"isBuyerMaker":true,"isBestMatch":true},{"id":2,"price":"414.15000000","qty":"0.02729000","quoteQty":"11.30215350","time":1597854349295,"isBuyerMaker":true,"isBestMatch":true},{"id":3,"price":"414.15000000","qty":"0.02733000","quoteQty":"11.31871950","time":1597854483275,"isBuyerMaker":true,"isBestMatch":true},{"id":4,"price":"414.15000000","qty":"0.02733000","quoteQty":"11.31871950","time":1597854687743,"isBuyerMaker":true,"isBestMatch":true},{"id":5,"price":"414.15000000","qty":"0.02733000","quoteQty":"11.31871950","time":1597854803719,"isBuyerMaker":true,"isBestMatch":true},{"id":6,"price":"414.15000000","qty":"0.02738000","quoteQty":"11.33942700","time":1597854865008,"isBuyerMaker":true,"isBestMatch":true},{"id":7,"price":"414.15000000","qty":"0.0273

[{'id': 0,
  'price': '386.26000000',
  'qty': '1.00000000',
  'quoteQty': '386.26000000',
  'time': 1596440526404,
  'isBuyerMaker': True,
  'isBestMatch': True},
 {'id': 1,
  'price': '414.15000000',
  'qty': '0.02714000',
  'quoteQty': '11.24003100',
  'time': 1597851105665,
  'isBuyerMaker': True,
  'isBestMatch': True},
 {'id': 2,
  'price': '414.15000000',
  'qty': '0.02729000',
  'quoteQty': '11.30215350',
  'time': 1597854349295,
  'isBuyerMaker': True,
  'isBestMatch': True},
 {'id': 3,
  'price': '414.15000000',
  'qty': '0.02733000',
  'quoteQty': '11.31871950',
  'time': 1597854483275,
  'isBuyerMaker': True,
  'isBestMatch': True},
 {'id': 4,
  'price': '414.15000000',
  'qty': '0.02733000',
  'quoteQty': '11.31871950',
  'time': 1597854687743,
  'isBuyerMaker': True,
  'isBestMatch': True},
 {'id': 5,
  'price': '414.15000000',
  'qty': '0.02733000',
  'quoteQty': '11.31871950',
  'time': 1597854803719,
  'isBuyerMaker': True,
  'isBestMatch': True},
 {'id': 6,
  'price':

In [48]:
#After checking the balance we send the ether to the account address of our profitsplitter.
#The profitsplitter should automatically divide the profits and send ether to each team member's wallet.

quantity = 7
#profit splitter address deployed on the Ropsten test network
address = '0xf7748a5b4B867293FeC8ec0c3d3f2dCAc29d7a2e'

#Note: This withdraw function will not work on the testnet but should work on the live binance api
withdraw(quantity)



b'<!DOCTYPE html>\n<html lang="en">\n<head>\n    <title>Binance Spot Test Network</title>\n    <link rel="shortcut icon" type="image/x-icon" href="/static/images/common/favicon.ico"/>\n    <style>\nhtml {\n    margin: 0;\n}\n\nbody {\n    display: flex;\n    flex-direction: column;\n    margin: 0;\n    padding: 0;\n    min-height: 100vh;\n    font-family: BinancePlex, Arial, PingFangSC-Regular, \'Microsoft YaHei\', sans-serif !important;\n}\n\n#header {\n    height: 64px;\n    background-color: #12161C;\n    margin: 0;\n    padding: 0;\n}\n\n#body {\n    display: flex;\n    flex: 1;\n    margin: 0;\n    padding: 0;\n}\n\n#content {\n    width: 800px;\n    margin-left: auto;\n    margin-right: auto;\n    line-height: 26px;\n}\n\n#footer {\n    background-color: #12161C;\n    font-size: 12px;\n    line-height: 16px;\n    color: #76808F;\n    text-align: center;\n    margin-top: 20px;\n    margin-bottom: 0;\n    margin-left: 0;\n    margin-right: 0;\n    padding: 12px;\n}\n\n#logo {\n    

b'<!DOCTYPE html>\n<html lang="en">\n<head>\n    <title>Binance Spot Test Network</title>\n    <link rel="shortcut icon" type="image/x-icon" href="/static/images/common/favicon.ico"/>\n    <style>\nhtml {\n    margin: 0;\n}\n\nbody {\n    display: flex;\n    flex-direction: column;\n    margin: 0;\n    padding: 0;\n    min-height: 100vh;\n    font-family: BinancePlex, Arial, PingFangSC-Regular, \'Microsoft YaHei\', sans-serif !important;\n}\n\n#header {\n    height: 64px;\n    background-color: #12161C;\n    margin: 0;\n    padding: 0;\n}\n\n#body {\n    display: flex;\n    flex: 1;\n    margin: 0;\n    padding: 0;\n}\n\n#content {\n    width: 800px;\n    margin-left: auto;\n    margin-right: auto;\n    line-height: 26px;\n}\n\n#footer {\n    background-color: #12161C;\n    font-size: 12px;\n    line-height: 16px;\n    color: #76808F;\n    text-align: center;\n    margin-top: 20px;\n    margin-bottom: 0;\n    margin-left: 0;\n    margin-right: 0;\n    padding: 12px;\n}\n\n#logo {\n    

In [49]:
#Then we check account holdings again to make sure the withdrawal was completed.
holdings = get_account_holdings()

#convert bytes object to string
holdings = holdings.decode("utf-8") 
    
#convert str to json
print(f"This is your account information and current holdings")
holdings = json.loads(holdings)
holdings

This is your account information and current holdings


{'makerCommission': 0,
 'takerCommission': 0,
 'buyerCommission': 0,
 'sellerCommission': 0,
 'canTrade': True,
 'canWithdraw': False,
 'canDeposit': False,
 'updateTime': 1598383158268,
 'accountType': 'SPOT',
 'balances': [{'asset': 'BNB',
   'free': '1000.00000000',
   'locked': '0.00000000'},
  {'asset': 'BTC', 'free': '0.99000000', 'locked': '0.00000000'},
  {'asset': 'BUSD', 'free': '5558.07298600', 'locked': '0.00000000'},
  {'asset': 'ETH', 'free': '110.90311000', 'locked': '0.00000000'},
  {'asset': 'LTC', 'free': '500.00000000', 'locked': '0.00000000'},
  {'asset': 'TRX', 'free': '500000.00000000', 'locked': '0.00000000'},
  {'asset': 'USDT', 'free': '10400.00000000', 'locked': '0.00000000'},
  {'asset': 'XRP', 'free': '50000.00000000', 'locked': '0.00000000'}],
 'permissions': ['SPOT']}

In [51]:
#Since we cannot use the withdraw function to send ether to our profitsplitter we use web3 instead.

from web3 import Web3, HTTPProvider
import json
    
from web3 import Web3 
# from web3.middleware import geth_poa_middleware
from dotenv import load_dotenv
from eth_account import Account
from pathlib import Path 
from getpass import getpass
load_dotenv()
import os

w3 = Web3(Web3.HTTPProvider("http://127.0.0.1:8545"))
# w3.middleware_onion.inject(geth_poa_middleware, layer=0)

private_key = os.getenv("private_key")


account_one = Account.from_key("secret_key")


def create_raw_tx(account, recipient, amount):
    """Sets up dict for a transaction"""

    gasEstimate = w3.eth.estimateGas({
        "from":account.address,
        "to":recipient,
        "value":amount
    })

    return {
        "from":account.address,
        "to":recipient,
        "value":amount,
        "gasPrice":w3.eth.gasPrice,
        "gas":gasEstimate,
        "nonce": w3.eth.getTransactionCount(account.address)
        }

def send_tx(account, recipient, amount):
    """Transforms raw_tx and sends on network using w3"""

    tx = create_raw_tx(account, recipient,amount)
    signed_tx = account.sign_transaction(tx)
    result = w3.eth.sendRawTransaction(signed_tx.rawTransaction)

    print(result.hex())
    return result.hex()


send_tx(account_one, "0x8AFbDa01e8eb1d90035c4820ae385c365C6E2f96", int(50*10e-18))

print(w3.eth.blockNumber)


print(w3.eth.getBalance("0x06297Fe891046c0Bfaaf3577259CEc070E66103a"))


Python-dotenv could not parse statement starting at line 11
Python-dotenv could not parse statement starting at line 15


Error: Non-hexadecimal digit found